In [1]:
import getpass
import os

password = getpass.getpass()
command = "sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null"
os.system('echo %s | %s' % (password, command))

KeyboardInterrupt: Interrupted by user

In [18]:
!sudo apt-get -y install openjdk-8-jre-headless
!pip install pyspark


Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u265-b01-0+deb9u1).
openjdk-8-jre-headless set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.
     |████████████████████████████████| 204.7 MB 16 kB/s s eta 0:00:01
     |████████████████████████████████| 198 kB 75.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=36fd05a2d0f1a3a372e51bdfbea19f17a942997e88137dc2e3ab9b2c35cf1f1c
  Stored in directory: /home/jupyter/.cache/pip/wheels/4e/c5/36/aef1bb711963a619063119cc032176106827a129c0be20e301
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [2]:
import os
import pandas as pd
import numpy as np
import re
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as F

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS #, Rating #, MatrixFactorizationModel

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
1

1

In [4]:
train_features = pd.read_csv("train_features.csv")
test_features = pd.read_csv("test_features.csv")

In [ ]:
train_features.head(1)

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length_x,genre_ids_x,...,language_y,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year
0,0,0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,...,52.0,102,215,47.4,528.0,1140,2791,5511,50.6,3.0


In [ ]:
def str2id(msno):
    res = re.sub('[a-zA-Z]','', msno)
    ret = str(re.sub('\W+','',res))
    if len(ret) > 6:
        return ret[:6]
    return ret
    


train_features['msno_id'] = train_features['msno'].apply(str2id)

In [ ]:
test_features['msno_id'] = test_features['msno'].apply(str2id)

In [ ]:
train_features['song_new_id'] = train_features['song_id'].apply(str2id)

In [ ]:
test_features['song_new_id'] = test_features['song_id'].apply(str2id)

In [ ]:
train_features['msno_id'] = train_features['msno_id'].fillna("0")
test_features['msno_id'] = test_features['msno_id'].fillna("0")
train_features['song_new_id'] = train_features['song_new_id'].fillna("0")
test_features['song_new_id'] = test_features['song_new_id'].fillna("0")
train_features['target'] = train_features['target'].fillna(0)

In [ ]:
train_features['source_system_tab'] = train_features['source_system_tab'].fillna('')
train_features['source_screen_name'] = train_features['source_screen_name'].fillna('')
train_features['source_type'] = train_features['source_type'].fillna('')
train_features['song_length_x'] = train_features['song_length_x'].fillna(0)
train_features['genre_ids_x'] = train_features['genre_ids_x'].fillna(0)
train_features['artist_name_x'] = train_features['artist_name_x'].fillna('')
train_features['lyricist_x'] = train_features['lyricist_x'].fillna('')
train_features['composer_x'] = train_features['composer_x'].fillna('')
train_features['language_x'] = train_features['language_x'].fillna(0)
train_features['name'] = train_features['name'].fillna('')
train_features['isrc'] = train_features['isrc'].fillna(0)
train_features['city'] = train_features['city'].fillna(0)
train_features['bd'] = train_features['bd'].fillna(0)
train_features['gender'] = train_features['gender'].fillna('')
train_features['registered_via'] = train_features['registered_via'].fillna(0)
train_features['registration_init_time'] = train_features['registration_init_time'].fillna('1901-01-01')
train_features['expiration_date'] = train_features['expiration_date'].fillna('1901-01-01')
train_features['repeat_count'] = train_features['repeat_count'].fillna(0)
train_features['play_count'] = train_features['play_count'].fillna(0)
train_features['repeat_percentage'] = train_features['repeat_percentage'].fillna(0)
train_features['repeat_count_artist'] = train_features['repeat_count_artist'].fillna(0)
train_features['play_count_artist'] = train_features['play_count_artist'].fillna(0)
train_features['repeat_count_msno'] = train_features['repeat_count_msno'].fillna(0)
train_features['play_count_msno'] = train_features['play_count_msno'].fillna(0)
train_features['repeat_percentage_msno'] = train_features['repeat_percentage_msno'].fillna(0)
train_features['song_year'] = train_features['song_year'].fillna(0)
#train_features['description'] = train_features['description'].fillna(0)

In [ ]:
test_features['source_system_tab'] = train_features['source_system_tab'].fillna('')
test_features['source_screen_name'] = train_features['source_screen_name'].fillna('')
test_features['source_type'] = train_features['source_type'].fillna('')
test_features['song_length_x'] = train_features['song_length_x'].fillna(0)
test_features['genre_ids_x'] = train_features['genre_ids_x'].fillna(0)
test_features['artist_name_x'] = train_features['artist_name_x'].fillna('')
test_features['lyricist_x'] = train_features['lyricist_x'].fillna('')
test_features['composer_x'] = train_features['composer_x'].fillna('')
test_features['language_x'] = train_features['language_x'].fillna(0)
test_features['name'] = train_features['name'].fillna('')
test_features['isrc'] = train_features['isrc'].fillna(0)
test_features['city'] = train_features['city'].fillna(0)
test_features['bd'] = train_features['bd'].fillna(0)
test_features['gender'] = train_features['gender'].fillna('')
test_features['registered_via'] = train_features['registered_via'].fillna(0)
test_features['registration_init_time'] = train_features['registration_init_time'].fillna('1901-01-01')
test_features['expiration_date'] = train_features['expiration_date'].fillna('1901-01-01')
test_features['repeat_count'] = train_features['repeat_count'].fillna(0)
test_features['play_count'] = train_features['play_count'].fillna(0)
test_features['repeat_percentage'] = train_features['repeat_percentage'].fillna(0)
test_features['repeat_count_artist'] = train_features['repeat_count_artist'].fillna(0)
test_features['play_count_artist'] = train_features['play_count_artist'].fillna(0)
test_features['repeat_count_msno'] = train_features['repeat_count_msno'].fillna(0)
test_features['play_count_msno'] = train_features['play_count_msno'].fillna(0)
test_features['repeat_percentage_msno'] = train_features['repeat_percentage_msno'].fillna(0)
test_features['song_year'] = train_features['song_year'].fillna(0)
#train_features['description'] = train_features['description'].fillna(0)


In [ ]:

train_features.shape

(7377418, 39)

In [17]:
train_features.dropna()
test_features.dropna()
train_features.shape

(7377418, 39)

In [15]:
train_features.repeat_percentage_msno.isna().sum()

0

In [47]:
test_features.head(5)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,artist_name_x,composer_x,lyricist_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length_y,genre_ids_y,artist_name_y,composer_y,lyricist_y,language_y,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,msno_id,song_new_id
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,explore,Explore,online-playlist,206471.0,359,Bastille,Dan Smith| Mark Crew,,52.0,Good Grief,GBUM71602854,1,0,,7,2012-01-02,2017-10-05,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0,102,215,47.4,528.0,1140,2791,5511,50.6,3.0,8773516,1725
1,1,1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-playlist,284584.0,1259,Various Artists,,,52.0,Lords of Cardboard,US3C69910183,13,24,female,9,2011-05-25,2017-09-11,320470.0,465,林俊傑 (JJ Lin),林俊傑,孫燕姿/易家揚,3.0,1,1,0.0,154799.0,303616,462,622,74.3,19.0,8773516,9752253
2,2,2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,my library,Local playlist more,local-playlist,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,,52.0,Hip Hop Is Dead(Album Version (Edited)),USUM70618761,13,24,female,9,2011-05-25,2017-09-11,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,Yu Takahashi,17.0,2,4,50.0,62.0,289,462,622,74.3,1.0,2270,8525304
3,3,3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,my library,Local playlist more,local-playlist,255512.0,1019,Soundway,Kwadwo Donkoh,,-1.0,Disco Africa,GBUQH1000063,13,24,female,9,2011-05-25,2017-09-11,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,NaN,52.0,1,1,0.0,1.0,1,462,622,74.3,2.0,164910,843549
4,4,4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,explore,Explore,online-playlist,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,,52.0,Sleep Without You,QM3E21606003,1,0,,7,2012-01-02,2017-10-05,197590.0,873,Yoga Mr Sound,Neuromancer,NaN,-1.0,150,412,36.4,161.0,427,2791,5511,50.6,3.0,164910,538


In [ ]:
temp_train = train_features[['msno_id', 'song_new_id', 'target']]
temp_train.head(5)

,msno_id,song_new_id,target
0,182789,7126,1
1,643790,8643,1
2,643790,77437,1
3,643790,287746,1
4,182789,366118,1


In [ ]:
temp_test = test_features[['id', 'msno_id', 'song_new_id']]


In [ ]:
temp_train.to_csv("temp2_train_features.csv")
temp_test.to_csv("temp2_test_features.csv")


In [ ]:
# Visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(18, 4))

import seaborn as sns
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})

np.set_printoptions(precision=4, suppress=True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# set random seed for reproducability
rnd_seed=42
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

In [ ]:
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("movie-recommender-pyspark")
         .getOrCreate())
spark

In [ ]:
sc = spark.sparkContext
sc
print (sc.defaultParallelism)

<SparkContext master=local appName=pyspark-shell>

1


In [ ]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

In [ ]:
print(sc.getConf().toDebugString())

spark.app.id=local-1599960709354
spark.app.name=pyspark-shell
spark.driver.host=python-20200612-104541.us-west1-b.c.winter-agency-280010.internal
spark.driver.port=34745
spark.executor.id=driver
spark.master=local
spark.rdd.compress=True
spark.serializer.objectStreamReset=100
spark.submit.deployMode=client
spark.submit.pyFiles=
spark.ui.showConsoleProgress=true


In [ ]:
train_features = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("mode", "DROPMALFORMED").load("temp2_train_features.csv")

In [ ]:
test_features = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("mode", "DROPMALFORMED").load("temp2_test_features.csv")

In [ ]:
train_features.limit(10).toPandas()

,_c0,msno_id,song_new_id,target
0,0,182789,7126,1
1,1,643790,8643,1
2,2,643790,77437,1
3,3,643790,287746,1
4,4,182789,366118,1
5,5,182789,351057,1
6,6,643790,13958,1
7,7,182789,5831,1
8,8,58387,6723,1
9,9,58387,731337,1


In [ ]:
1

1

In [ ]:
train_features.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- msno_id: integer (nullable = true)
 |-- song_new_id: integer (nullable = true)
 |-- target: integer (nullable = true)



In [ ]:
train_features = train_features.drop("_c0", "Unnamed: 0", "language_y", "lyricist_y", "composer_y", "genre_ids_y",
                  "song_length_y")

test_features = test_features.drop("_c0", "Unnamed: 0", "language_y", "lyricist_y", "composer_y", "genre_ids_y",
                  "song_length_y")

In [141]:
(print("Number of songs with at least one rating strictly higher than 1000: {0}".format(test_features.filter('play_count > 1000')
                                                                                     .select('song_id')
                                                                                     .distinct()
                                                                                     .count())))

Number of songs with at least one rating strictly higher than 1000: 1072


In [ ]:
train_features.head(1)

[Row(msno_id=182789, song_new_id=7126, target=1)]

In [99]:

# lm = (sns.lmplot(x='msno', y='song_id', hue='play_count', 
#                  data=test_features.toPandas(), 
#                  fit_reg=False, size=10, aspect=2, 
#                  palette=sns.palplot(sns.diverging_palette(250, 15, n=5, center="dark"))))

In [ ]:
# def str2id(msno):
#     res = re.sub('[a-zA-Z]','', msno)
#     ret = str(re.sub('\W+','',res))
#     if len(ret) > 7:
#         return ret[:7]
#     return ret
    


# func = udf(lambda x: str2id(x), IntegerType())

# #train_df['msno_id'] = train_df['msno'].apply(str2id)
# train_features.withColumn("msno_id",func(train_features.msno))
# train_features.withColumn("song_new_id",func(train_features.song_id))


In [ ]:
#因为训练集太大太大，难以训练
train_features.withColumn("target", train_features["target"].cast(IntegerType()))
train_df, test_df = train_features.randomSplit([0.8, 0.2], seed=rnd_seed)

DataFrame[msno_id: int, song_new_id: int, target: int]

In [ ]:
train_df = train_df.withColumn("target", train_df["target"].cast(IntegerType()))
test_df = test_df.withColumn("target", test_df["target"].cast(IntegerType()))

In [42]:

# we will cache the data because we will be using this dataframes over and over again
train_df.cache()
test_df.cache()

DataFrame[msno_id: int, song_new_id: int, target: int]

DataFrame[msno_id: int, song_new_id: int, target: int]

In [43]:
train_df.printSchema()

root
 |-- msno_id: integer (nullable = true)
 |-- song_new_id: integer (nullable = true)
 |-- target: integer (nullable = true)



In [ ]:
train_features.head(1)

[Row(msno_id=182789, song_new_id=7126, target=1)]

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(rank=10, maxIter=10, regParam=0.01, userCol='msno_id', itemCol='song_new_id', ratingCol='target', coldStartStrategy="drop", seed=rnd_seed)



In [84]:
from pyspark.sql import functions
from pyspark.mllib.evaluation import RankingMetrics, RegressionMetrics
#train_df.withColumn("msno_id", functions.lit(0))

In [ ]:
import gc
gc.collect()

109

In [38]:
from pyspark.sql.functions import isnull, isnan
train_df.select(isnull("song_new_id").alias("0")).show()

+-----+
|    0|
+-----+
| true|
| true|
| true|
| true|
| true|
| true|
| true|
| true|
| true|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
+-----+
only showing top 20 rows



In [41]:
#train_df["msno_id"].combine_first(0)

In [ ]:
train_features.limit(10).toPandas()

,msno_id,song_new_id,target
0,182789,7126,1
1,643790,8643,1
2,643790,77437,1
3,643790,287746,1
4,182789,366118,1
5,182789,351057,1
6,643790,13958,1
7,182789,5831,1
8,58387,6723,1
9,58387,731337,1


In [ ]:
#train_df = train_df.orderBy("msno_id")
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
train_df.limit(10).toPandas()

,msno_id,song_new_id,target
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,1
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,0


In [ ]:
model = als.fit(train_df)

In [45]:
model.userFactors.show(10, 100)

+---+----------------------------------------------------------------------------------------------------+
| id|                                                                                            features|
+---+----------------------------------------------------------------------------------------------------+
|  0|[-0.025261449, -0.3200597, 0.13558546, -0.06688132, -0.36999127, -0.37474155, -0.47915238, -0.249...|
| 10|[-0.38584155, -0.5800418, 0.12796, -0.03428915, 0.010083084, -0.64424795, -0.4288808, -0.14846393...|
| 20|[-0.110638276, -0.43918008, 0.26699138, 0.2653487, -0.4436625, -0.68173647, -0.23061687, -0.06858...|
| 30|[-0.3659993, -0.27386227, -0.027479652, 0.35194445, -0.19346803, -0.5503287, -0.19324715, -0.4417...|
| 40|[-0.218055, 0.050975718, -0.2741511, 0.0060462668, -0.31619093, -0.4281153, -0.22927262, -0.37442...|
| 50|[-0.1725919, -0.17967756, -0.08847102, -0.14216883, -0.42307615, -0.8067499, -0.2566834, -0.38349...|
| 60|[-0.118093126, -0.517598, 0.1779

In [ ]:
model.userFactors.count()

27805

In [48]:
model.itemFactors.show(10, 100)

+---+----------------------------------------------------------------------------------------------------+
| id|                                                                                            features|
+---+----------------------------------------------------------------------------------------------------+
|  0|[-0.08059619, -0.14277264, 0.002137795, 0.04637022, -0.32817626, -0.35285893, -0.17084545, -0.324...|
| 10|[-0.009540478, -0.2703578, 0.072347164, -0.19129835, -0.18653986, -0.22790945, -0.13785708, -0.42...|
| 20|[-0.32017204, -0.3497971, -0.04570556, -0.068731226, -0.2132581, -0.07885092, -0.22913638, -0.353...|
| 30|[-0.018630382, -0.29803267, 0.007969981, -0.07298389, -0.15459387, -0.30189875, -0.22593027, -0.3...|
| 40|[-0.031342316, -0.2743831, 0.024820112, -0.0019409717, -0.32987043, -0.41848636, -0.15371953, -0....|
| 50|[-0.13556553, -0.2704415, -0.08949764, -0.11744851, -0.17650323, -0.051439345, -0.2937678, -0.421...|
| 60|[-0.02466895, -0.22568442, 0.013

In [55]:
paramGrid = (ParamGridBuilder()
             .addGrid(als.rank, [1, 5, 10])
             .addGrid(als.maxIter, [20])
             .addGrid(als.regParam, [0.05, 0.1, 0.5])
             .build())

In [56]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='target', metricName='rmse')

In [59]:
crossval = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

In [ ]:
cvModel = crossval.fit(train_df)

In [63]:
1

1

In [64]:
from pyspark.ml.recommendation import ALS

def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS().setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

2

In [65]:
best_model = cvModel.bestModel

In [49]:
test_df.sample(fraction=0.01).limit(10).show(10)

+-------+-----------+------+
|msno_id|song_new_id|target|
+-------+-----------+------+
|      0|      29513|     0|
|      0|      84309|     1|
|      0|      86508|     1|
|      0|     129488|     1|
|      0|     170477|     0|
|      0|     206968|     1|
|      0|     250958|     1|
|      0|     596355|     0|
|      0|     642897|     1|
|      0|     643659|     0|
+-------+-----------+------+



In [71]:
music_pred = cvModel.transform(test_df)
music_pred.show()
#model.recommendProductsForUsers(643790, 287746)

+-------+-----------+------+-----------+
|msno_id|song_new_id|target| prediction|
+-------+-----------+------+-----------+
| 565154|        148|     0|  0.2910931|
| 279532|        148|     1| 0.30200347|
| 489334|        148|     1| 0.20511226|
| 898551|        148|     0|0.101649076|
| 292001|        148|     0| 0.28564808|
| 700446|        148|     0|  0.5206393|
| 847780|        148|     0| 0.19588542|
| 232047|        148|     0| 0.20670481|
|    691|        148|     1|   0.317098|
| 405872|        148|     0| 0.30772084|
|  12176|        148|     1|  0.5971481|
|  41563|        148|     0|  0.3444055|
| 426116|        148|     1|  0.8897885|
| 382227|        148|     0|  0.2813537|
|  41757|        148|     0| 0.34624657|
| 627940|        148|     1| 0.12110627|
| 421445|        148|     0| 0.17576729|
|  10548|        148|     0| 0.38657314|
| 606282|        148|     0| 0.23770678|
|  66259|        148|     0| 0.25451034|
+-------+-----------+------+-----------+
only showing top

In [73]:
music_pred.filter("prediction < 1.0 or prediction > 0.5").show(100)

+-------+-----------+------+------------+
|msno_id|song_new_id|target|  prediction|
+-------+-----------+------+------------+
| 565154|        148|     0|   0.2910931|
| 279532|        148|     1|  0.30200347|
| 489334|        148|     1|  0.20511226|
| 898551|        148|     0| 0.101649076|
| 292001|        148|     0|  0.28564808|
| 700446|        148|     0|   0.5206393|
| 847780|        148|     0|  0.19588542|
| 232047|        148|     0|  0.20670481|
|    691|        148|     1|    0.317098|
| 405872|        148|     0|  0.30772084|
|  12176|        148|     1|   0.5971481|
|  41563|        148|     0|   0.3444055|
| 426116|        148|     1|   0.8897885|
| 382227|        148|     0|   0.2813537|
|  41757|        148|     0|  0.34624657|
| 627940|        148|     1|  0.12110627|
| 421445|        148|     0|  0.17576729|
|  10548|        148|     0|  0.38657314|
| 606282|        148|     0|  0.23770678|
|  66259|        148|     0|  0.25451034|
|  40020|        148|     0|  0.44

In [74]:
music_rating_preds = (music_pred
                      .withColumn("prediction_rounded", F.rint('prediction').cast(FloatType()))
                      .withColumn("target", col('target').cast(FloatType()))
                     )
music_rating_preds.cache()

DataFrame[msno_id: int, song_new_id: int, target: float, prediction: float, prediction_rounded: float]

In [75]:
RMSE = evaluator.evaluate(music_rating_preds)

In [76]:
print("Root Mean Squared Error = {0}".format(RMSE))

Root Mean Squared Error = 0.4550337605391539


In [77]:

# RMSE with rating rounded column
RegressionEvaluator(predictionCol='prediction_rounded', labelCol='target', metricName='rmse').evaluate(music_rating_preds)

0.5620356021749016

In [ ]:
from pyspark.mllib.evaluation import RankingMetrics, RegressionMetrics

In [79]:
perUserActual = (music_rating_preds
                 .where(col("target") > 0.7)
                 .groupBy("msno_id")
                 .agg(F.collect_set("song_new_id").alias("music")))

In [80]:
perUserActual.show()

+-------+--------------------+
|msno_id|               music|
+-------+--------------------+
|  14832|[48609, 952262, 8...|
|  32592|[166021, 424333, ...|
| 302124|[601176, 78190, 4...|
| 716163|[568270, 30, 9541...|
| 740678|[48609, 60, 11108...|
| 882285|[424333, 28545, 3...|
|   2366|[481385, 8068, 17...|
|  73091|[81805, 844354, 4...|
| 188125|       [897, 674748]|
| 228408|[1199, 279323, 74...|
| 251742|[0, 864927, 39304...|
| 327814|[46852, 9380, 501...|
| 328324|[12704, 889390, 3...|
| 470179|[864927, 731008, ...|
| 474305|[5039, 7384, 2850...|
| 492123|[48609, 514391, 4...|
| 551582|[246184, 239491, ...|
| 597517|[2063, 78190, 963...|
| 632365|[760996, 93937, 5...|
| 679597|[33547, 198920, 9...|
+-------+--------------------+
only showing top 20 rows



In [81]:

perUserPredictions = (music_rating_preds.orderBy(["msno_id", "prediction"], ascending=[True, False])
                 .groupBy("msno_id")
                 .agg(F.collect_set("song_new_id").alias("music")))

perUserPredictions.show()


+-------+--------------------+
|msno_id|               music|
+-------+--------------------+
|    148|[98385, 3987, 293...|
|    463|[610, 0, 763363, ...|
|    471|[1902, 901099, 68...|
|    496|[97775, 100099, 3...|
|   1088|[530588, 443753, ...|
|   1591|[847584, 256978, ...|
|   1829|[48609, 386629, 5...|
|   1959|[486988, 557208, ...|
|   2142|[11143, 89638, 49...|
|   2366|[593997, 565832, ...|
|   2659|[19376, 510079, 9...|
|   2866|[883869, 82176, 2...|
|   3997|[0, 97362, 8320, ...|
|   4818|[453464, 283475, ...|
|   4900|[0, 972544, 46852...|
|   4935|[14680, 116972, 605]|
|   7754|[33411, 236729, 8...|
|   7880|[76688, 10680, 92...|
|   8389|[565832, 439742, ...|
|   8592|     [375062, 67529]|
+-------+--------------------+
only showing top 20 rows



In [82]:
perUserActualvPred = (perUserActual.join(perUserPredictions, ["msno_id"]).rdd
                      .map(lambda row: (row[1], row[2][:15])))

In [85]:

ranks = RankingMetrics(perUserActualvPred)

In [86]:
ranks

In [87]:
ranks.meanAveragePrecision

0.410300350722384

In [88]:
ranks.precisionAt(5)

0.6010418057967143

In [89]:
#take lots of time
# top 3 movies recommended for each user
user_recs = best_model.recommendForAllUsers(3)

In [90]:
user_recs.show(truncate=False)

+-------+------------------------------------------------------------------+
|msno_id|recommendations                                                   |
+-------+------------------------------------------------------------------+
|148    |[[667587, 0.56942385], [86750, 0.56942385], [839881, 0.5307072]]  |
|463    |[[326260, 1.208981], [49414, 1.208981], [307117, 1.2072794]]      |
|471    |[[839881, 0.6772567], [842900, 0.662526], [410034, 0.6561722]]    |
|496    |[[513450, 1.0301806], [326260, 1.0258677], [49414, 1.0258677]]    |
|1088   |[[513450, 0.91686845], [984190, 0.8857839], [348837, 0.852147]]   |
|1591   |[[49964, 1.4699262], [410034, 1.4618485], [44407, 1.4030604]]     |
|1829   |[[488959, 0.9008638], [313146, 0.8915163], [606821, 0.89049286]]  |
|1959   |[[384420, 1.3532628], [676952, 1.3355956], [67659, 1.280351]]     |
|2142   |[[305992, 0.98327714], [139261, 0.98327714], [907603, 0.98327714]]|
|2366   |[[307117, 1.5458423], [842900, 1.5193088], [839881, 1.5192887]]   |

In [135]:
user_recs

DataFrame[msno_id: int, recommendations: array<struct<song_new_id:int,rating:float>>]

In [91]:
user_recs_one = user_recs.where(user_recs.msno_id == 1959).cache()
user_recs_one.show(truncate=False)

+-------+-------------------------------------------------------------+
|msno_id|recommendations                                              |
+-------+-------------------------------------------------------------+
|1959   |[[384420, 1.3532628], [676952, 1.3355956], [67659, 1.280351]]|
+-------+-------------------------------------------------------------+



In [136]:
user_recs_one.select("msno_id", "recommendations.song_new_id", "recommendations.rating").collect()

[Row(msno_id=1959, song_new_id=[384420, 676952, 67659], rating=[1.3532627820968628, 1.3355956077575684, 1.2803510427474976])]

In [141]:
user_recs_one = user_recs_one.toPandas()

In [142]:
user_recs_one

,msno_id,recommendations
0,1959,"[(384420, 1.3532627820968628), (676952, 1.3355956077575684), (67659, 1.2803510427474976)]"


In [98]:
#hybrid

In [99]:
cosine_sim = np.loadtxt('cosine_sim.txt',delimiter=',')

In [100]:
cosine_sim.shape

(837500, 500)

44

In [145]:
def hybrid(userId, title):
    
    idxs = get_index_by_name(title)    
    sim_scores = get_cosine_sim_scores2(idxs)
    movies = remove_blank(sim_scores)
    movies = getFrame_by_indexs(movies)
    

    movies = movies.drop(['id','index'], axis=1)
    movies['msno'] = userId
    movies = change_data_type(movies)
    
    movies['est'] = gbm.predict(movies)
    #movies['est'] = movies['song_id'].apply(lambda x: algo.predict(userId, x).est)
                                 
#     movies = movies.sort_values('est', ascending=False)
#     return movies.head(10)
    return movies

def change_data_type(music):
    drop_list2 = ['Unnamed: 0', 'Unnamed: 0.1']
    music = music.drop(drop_list2,axis=1)
    for col in music.columns:
        if music[col].dtype == object:
            #test_features[col] = train_features[col].astype('category')
            music[col] = music[col].astype('category')
    return music

def getFrame_by_indexs(movies):
    indexs = []
    for i, v in movies.items():
        indexs.append(i)
    indexs = sorted(indexs)
    begin = new_train_features.iloc[indexs[-1]:indexs[-1]+1]
    for i in range(len(indexs)-1):
        results = new_train_features.iloc[indexs[i]:indexs[i]+1]
        begin = pd.concat([begin, results])

    return begin



def get_index_by_name(name):
    specific_songs = new_train_features.loc[new_train_features.name.str.contains(name)]
    #specific_songs = test_features.loc[test_features.name.str.contains(name)]
    dd = specific_songs.head(5)
    #print(dd.index)
    return dd.index

def get_scores(slist):
    res = []
    for i in slist:
        if i != 0:
            res.append(i)
    return res

def get_cosine_sim_scores2(idxs):
    res = []
    for i in idxs:
        sim_scores = list(enumerate(cosine_sim[i]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        movie_indices = [i[0] for i in sim_scores]
        temp = titles.iloc[movie_indices]
        res.append(temp)

    return res

def get_cosine_sim_scores(idx):
    res = []
    for i in range(len(cosine_sim_list)):
        if len(consine_sim_list[i]) > idx:
            temp = cosine_sim_list[i][idx]
            res.append(temp)
            
        #temp = map(lambda x: get_scores(x), cosine_sim_list)
    sim_scores = list(enumerate(res))
    #print(sim_scores)
    return sim_scores


In [113]:
def get_recommendation(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]    
    
        
def get_recommendations2(title):
    idxs = get_index_by_name(title)    
    sim_scores = get_cosine_sim_scores2(idxs)
    return sim_scores

In [223]:
new_train_features = pd.read_csv('test_features.csv')

In [224]:
new_train_features.drop('song_length_y',axis = 1 , inplace =True)
new_train_features.drop('genre_ids_y',axis = 1 , inplace =True)
new_train_features.drop('artist_name_y',axis = 1 , inplace =True)
new_train_features.drop('composer_y',axis = 1 , inplace =True)
new_train_features.drop('lyricist_y',axis = 1 , inplace =True)
new_train_features.drop('language_y',axis = 1 , inplace =True)
new_train_features.drop('lyricist_x',axis = 1 , inplace =True)
new_train_features.drop('source_type',axis = 1 , inplace =True)


In [225]:
1

1

In [226]:
new_train_features['description'] = new_train_features['name'] + new_train_features['artist_name_x']
new_train_features['name'] = new_train_features['name'].fillna('')

In [227]:
new_train_features.head(1)

,Unnamed: 0,Unnamed: 0.1,id,msno,song_id,source_system_tab,source_screen_name,song_length_x,genre_ids_x,artist_name_x,composer_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,3.0,愛其實很殘忍,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0,愛其實很殘忍梁文音 (Rachel Liang)


In [228]:
new_train_features = new_train_features.reset_index()
titles = new_train_features.name


In [229]:
titles.shape

(837941,)

In [230]:
recom = get_recommendations2('my love')

In [231]:
def remove_blank(recom):
    res = dict()
    for r in recom:
        for i, v in r.items():
            temp = ' '.join(v.split())
            if temp != '':
                res[i] = temp
    return res
remove_blank(recom)

{11: 'koi',
 13: '伊是阮老爸 (My Dad)',
 14: '決絕的信仰 (The Resolute Belief)',
 15: '釘子花',
 37: 'ho hay yan (喔嗨洋)',
 63: '神秘嘉賓 (Mystery Guest)',
 73: '你就不要想起我',
 75: '18',
 154: 'On My Way',
 164: '凉凉',
 165: "Miss Movin' On",
 174: '淘汰(國)',
 191: 'Get you！',
 233: '當你',
 257: 'The Best Is Yet To Come',
 259: 'Se Me Olivido Tu Nombre(忘了妳是誰)',
 263: 'Besame Mucho',
 269: 'Never Be Like You',
 345: 'Love on the Weekend',
 349: 'Crush',
 371: "Thinking 'Bout You",
 444: 'Take Me To Church',
 481: '女人花',
 495: '陪你 (With you)',
 499: 'Never Say Goodbye',
 394: 'Adventure Of A Lifetime',
 494: 'Faded',
 308: 'PLAY 我呸',
 334: 'This Love',
 496: 'Nagareboshi',
 25: '聽說 (Listen and Say)',
 103: '全世界的人都知道',
 117: '新年未老 (New Year)',
 162: 'Sucker For Pain',
 182: '小天使 (Little Angel)',
 289: '還是會寂寞',
 290: 'Run The Show (featuring Shaka Dee)',
 291: 'Faith',
 318: 'Relaxing Rain',
 322: 'White Noise for Deep Sleep Short',
 323: '該忘的日子 (I Am Moving On)',
 325: 'Radioactive (電影「宿主」、電玩「刺客教條3」主題曲、NBA 2012-201

In [149]:
import lightgbm as lgb
gbm = lgb.Booster(model_file='model.txt')
    

In [232]:
new_train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837941 entries, 0 to 837940
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   837941 non-null  int64  
 1   Unnamed: 0              837941 non-null  int64  
 2   Unnamed: 0.1            837941 non-null  int64  
 3   id                      837941 non-null  int64  
 4   msno                    837941 non-null  object 
 5   song_id                 837941 non-null  object 
 6   source_system_tab       835145 non-null  object 
 7   source_screen_name      787802 non-null  object 
 8   song_length_x           837934 non-null  float64
 9   genre_ids_x             826839 non-null  object 
 10  artist_name_x           837934 non-null  object 
 11  composer_x              636468 non-null  object 
 12  language_x              837925 non-null  float64
 13  name                    837941 non-null  object 
 14  isrc                

In [243]:
def new_hybrid(msno_id, title):
    new_msno_id = str2id(msno_id)
    user_recs_one = user_recs.where(user_recs.msno_id == new_msno_id).cache()
    user_recs_one.show(truncate=False)
    user_recs_one = user_recs_one.toPandas()  
    
    idxs = get_index_by_name(title)    
    sim_scores = get_cosine_sim_scores2(idxs)
    music = remove_blank(sim_scores)
    music = getFrame_by_indexs(music)
    #"Unnamed: 0", "Unnamed: 0.1",
    music = music.drop(['id','index'], axis=1)
    music['msno'] = msno_id
    music = change_data_type(music)
    
    music['est'] = gbm.predict(music)    
    
    music['msno_id'] = music['msno'].apply(str2id)
    
    #res = pd.merge(user_recs_one, music, how='left', on='msno_id')
    return user_recs_one, music

In [ ]:
#we can combine music and music all, merging them together to make a recommendation list

In [244]:
music, music_all = new_hybrid("91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=",'my love')

+-------+--------------------------------------------------------------+
|msno_id|recommendations                                               |
+-------+--------------------------------------------------------------+
|919574 |[[590906, 0.4549486], [958626, 0.438386], [351703, 0.4266878]]|
+-------+--------------------------------------------------------------+



In [245]:
music

,msno_id,recommendations
0,919574,"[(590906, 0.4549486041069031), (958626, 0.4383859932422638), (351703, 0.4266878068447113)]"


In [246]:
music_all

,msno,song_id,source_system_tab,source_screen_name,song_length_x,genre_ids_x,artist_name_x,composer_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year,description,est,msno_id
499,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,W+gqrO3dhhuHUO0wAZTSobfk0Vf207+MDDnxd9DNY9g=,discover,Online playlist more,260527.0,444,Hwayobi,Sung Il Park,31.0,Never Say Goodbye,KRA0C1700001,1,0,NaN,7,2016-06-13,2017-09-13,41.0,105.0,39.0,51.0,131.0,50.0,97.0,51.5,3.0,Never Say GoodbyeHwayobi,0.212979,919574
0,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,3.0,愛其實很殘忍,TWUM71400047,1,0,NaN,7,2016-02-19,2017-09-18,350.0,694.0,50.4,7102.0,13654.0,52.0,142.0,36.6,2.0,愛其實很殘忍梁文音 (Rachel Liang),0.374245,919574
1,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,320470.0,465,林俊傑 (JJ Lin),林俊傑,3.0,她說,TWB671005201,1,0,NaN,7,2016-02-19,2017-09-18,3804.0,6090.0,62.5,67621.0,115325.0,52.0,142.0,36.6,2.0,她說林俊傑 (JJ Lin),0.453118,919574
2,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,17.0,subarashiki nichijo,JPWP01070260,1,0,NaN,4,2016-11-17,2016-11-24,2.0,5.0,40.0,286.0,989.0,19.0,132.0,14.4,2.0,subarashiki nichijoYu Takahashi (高橋優),0.048399,919574
3,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,52.0,Hold Me| Thrill Me| Kiss Me| Kill Me,GBAAN0201228,3,30,male,9,2007-07-25,2017-04-30,7.0,31.0,22.6,228.0,698.0,115.0,388.0,29.6,0.0,Hold Me| Thrill Me| Kiss Me| Kill MeU2,0.103330,919574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,g7l+2tpCTxb6MgdBUOrGQvr61PIjZXyN0UhXTTgz4YU=,my library,Local playlist more,231026.0,465,唐禹哲 (Danson Tang),林邁可,3.0,最愛還是你,TWD950729103,13,20,male,7,2015-11-15,2017-05-14,234.0,415.0,56.4,1534.0,3518.0,522.0,632.0,82.6,1.0,最愛還是你唐禹哲 (Danson Tang),0.551873,919574
494,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=,my library,Local playlist more,212750.0,1616|1609,Alan Walker,Alan Walker| Jesper Borgen| Anders Froen| Gunnar Greve,52.0,Faded,NOG841549010,3,34,female,9,2014-11-16,2017-09-30,7903.0,10565.0,74.8,23950.0,36781.0,178.0,289.0,61.6,3.0,FadedAlan Walker,0.854559,919574
495,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,G0rUz0kguI35yzj95LX2BXI30PqCcJJtPNfirR0BhIs=,discover,Online playlist more,260806.0,458,范瑋琪 (Christine Fan),饒善強,3.0,陪你 (With you),TWD251750501,1,0,NaN,7,2013-09-23,2017-09-12,3410.0,4834.0,70.5,14841.0,28719.0,97.0,210.0,46.2,3.0,陪你 (With you)范瑋琪 (Christine Fan),0.741872,919574
496,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,AGZKkV4V1Tbm881nAs6lnpncnN0NpMypfFxYWbPWQsk=,my library,Local playlist more,303464.0,465,Sayaka Shionoya (塩ノ谷早耶香),Erik Lidbom,17.0,Nagareboshi,JPKI01517963,3,34,female,9,2014-11-16,2017-09-30,8.0,20.0,40.0,128.0,324.0,178.0,289.0,61.6,3.0,NagareboshiSayaka Shionoya (塩ノ谷早耶香),0.212355,919574
